In [1]:
from xpinyin import Pinyin
import time
pin = Pinyin()

### Load File

In [2]:
# Parse the source
with open('dataset/THUOCL_chengyu.txt', 'r', encoding='utf-8') as source:
    idioms = [i for i in map(lambda x: x.split()[0], source.readlines()) if len(i) <= 4]

nodes = set([pin.get_pinyin(i[0]) for i in idioms] + [pin.get_pinyin(i[-1]) for i in idioms])
edges = [{"src": pin.get_pinyin(i[0]), "dst": pin.get_pinyin(i[-1]), "idiom": i} for i in idioms]

#### Create Graph

In [3]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687", password="12345678")

cql = "CREATE \n" + ",".join("(%s : Node {char: \"%s\"})\n"%(n, n) for n in nodes)
for e in edges:
    cql += ",(%s)-[:idiom {idiom: '%s'}]->(%s)\n"%(e['src'], e["idiom"], e['dst'])

start = time.time()
graph.run("match (n) detach delete n")
graph.run(cql)
print("created graph, time elapsed: %.2fs"%(time.time() - start))

created graph, time elapsed: 9.47s


___
### Query Graph
##### Find next word

In [4]:
next_word = lambda word: graph.run("match ()-[:idiom {idiom: '%s'}]->()-[e:idiom]->() return e.idiom"%word)

df_next = next_word("坚定不移").to_data_frame()

print("found %i idioms"%len(df_next))
df_next.head()

found 376 idioms


,e.idiom
0,一念之差
1,一五一十
2,一马当先
3,以古为镜
4,依然如故


##### Find path

In [5]:
def find_path(idiom, length, limit):
    cql = "match ()-[e:idiom {idiom: '%s'}]->()"%idiom
    path = "".join(["-[e%i:idiom]->()"%(i + 1) for i in range(length)])
    ret = ", ".join(["e%i.idiom as i%i"%(i + 1, i + 1) for i in range(length)])
    return graph.run(cql + path + " return e.idiom as i0, " + ret + " limit %i"%limit).to_data_frame()

In [6]:
start = time.time()
df_paths = find_path(idiom="坚定不移", length=5, limit=100000)

print("found %i paths, time elapsed: %.2fs"%(len(df_paths), time.time() - start))
df_paths.head(10)

found 100000 paths, time elapsed: 3.61s


,i0,i1,i2,i3,i4,i5
0,坚定不移,一念之差,差强人意,以讹传讹,饿虎扑食,事缓则圆
1,坚定不移,一五一十,实心实意,以讹传讹,饿虎扑食,事缓则圆
2,坚定不移,一五一十,诗情画意,以讹传讹,饿虎扑食,事缓则圆
3,坚定不移,一五一十,时移世易,以讹传讹,饿虎扑食,事缓则圆
4,坚定不移,一五一十,时移势易,以讹传讹,饿虎扑食,事缓则圆
5,坚定不移,一五一十,矢志不移,以讹传讹,饿虎扑食,事缓则圆
6,坚定不移,一五一十,始终如一,以讹传讹,饿虎扑食,事缓则圆
7,坚定不移,一马当先,弦外之意,以讹传讹,饿虎扑食,事缓则圆
8,坚定不移,以古为镜,惊叹不已,以讹传讹,饿虎扑食,事缓则圆
9,坚定不移,依然如故,顾名思义,以讹传讹,饿虎扑食,事缓则圆
